# AutoGen Alapvető Példa

Ebben a kódmintában az [AutoGen](https://aka.ms/ai-agents/autogen) AI keretrendszert fogod használni egy alapvető ügynök létrehozásához.

Ennek a példának a célja, hogy bemutassa azokat a lépéseket, amelyeket később a további kódmintákban fogunk alkalmazni a különböző ügynöki minták megvalósítása során.


## Importálja a szükséges Python csomagokat


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Hozza létre az Ügyfelet

Ebben a példában a [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) szolgáltatást fogjuk használni az LLM eléréséhez.

A `model` értéke `gpt-4o-mini`. Próbálja ki, hogy megváltoztatja a modellt egy másikra, amely elérhető a GitHub Models piacterén, hogy lássa a különböző eredményeket.

Gyors tesztként csak egy egyszerű kérdést futtatunk - `Mi Franciaország fővárosa`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Az ügynök meghatározása

Most, hogy beállítottuk a `client`-et és megerősítettük, hogy működik, hozzunk létre egy `AssistantAgent`-et. Minden ügynökhöz hozzárendelhető:  
**név** - Egy rövid név, amely hasznos lesz több ügynököt érintő folyamatokban való hivatkozáskor.  
**model_client** - Az a kliens, amelyet az előző lépésben hoztál létre.  
**tools** - Az ügynök által elérhető eszközök, amelyekkel egy feladatot elvégezhet.  
**system_message** - Az a metaprompt, amely meghatározza az LLM feladatát, viselkedését és hangnemét.  

A rendszerüzenetet megváltoztathatod, hogy lásd, hogyan reagál az LLM. Az `tools` témakört a 4. leckében fogjuk tárgyalni.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Az ügynök futtatása

Az alábbi függvény futtatja az ügynököt. Az `on_message` metódust használjuk arra, hogy frissítsük az ügynök állapotát az új üzenettel.

Ebben az esetben az állapotot egy új felhasználói üzenettel frissítjük, amely így szól: `"Tervezd meg nekem egy nagyszerű, napos vakációt"`.

Megváltoztathatod az üzenet tartalmát, hogy megnézd, hogyan reagál másképp az LLM.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Fontos információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
